# Imports

In [1]:
import csv
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Initialize Firestore

In [2]:
# Initialize Firebase Admin SDK with provided firebaseConfig
cred = credentials.Certificate("./firebase.json")
firebase_admin.initialize_app(cred)

# Function to convert a collection to csv

In [3]:
def export_firestore_collection_to_csv(collection_name, csv_file_path):
    # Get all documents from the collection
    docs = firestore.client().collection(collection_name).stream()

    # Define the desired order of columns (excluding createdAt)
    desired_order = ['email', 'firstName', 'lastName', 'nickname', 'phone', 'graduationYear', 'footPref', 'position']

    # Open CSV file for writing
    with open(csv_file_path, 'w', newline='') as csvfile:
        # Initialize a set to store all field names encountered
        fieldnames = set()

        # Iterate over all documents to collect field names
        for doc in docs:
            doc_data = doc.to_dict()
            fieldnames.update(doc_data.keys())

        # Sort the fieldnames so that the desired order comes first (excluding createdAt)
        fieldnames = sorted(fieldnames, key=lambda x: desired_order.index(x) if x in desired_order else len(desired_order))

        # Remove the 'createdAt' column from the fieldnames list
        fieldnames.remove('createdAt')

        # Write data to CSV
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        # Reset the stream to write data again
        docs = firestore.client().collection(collection_name).stream()

        for doc in docs:
            doc_data = doc.to_dict()
            # Exclude 'createdAt' from the document data
            doc_data.pop('createdAt', None)
            writer.writerow(doc_data)


# Using conversion on 'users' column

In [4]:
export_firestore_collection_to_csv('users', 'users.csv')
